<a href="https://colab.research.google.com/github/Sophie1205/pythoncourse/blob/main/EX05_04_Reply_Template_Messages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q flask pyngrok flask_ngrok line-bot-sdk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.1/776.1 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.6/165.6 kB 11.3 MB/s eta 0:00:00


In [3]:

# for colab
from google.colab import userdata
from pyngrok import ngrok
from flask_ngrok import run_with_ngrok
def ngrok_start():
    ngrok.set_auth_token(userdata.get('NGROK_AUTHTOKEN'))
    ngrok.connect(5000)
    run_with_ngrok(app)

from flask import Flask, request, abort

from linebot.v3 import WebhookHandler
from linebot.v3.exceptions import InvalidSignatureError
from linebot.v3.webhooks import MessageEvent, TextMessageContent
from linebot.v3.messaging import ( Configuration, ApiClient, MessagingApi,
    ReplyMessageRequest, TextMessage, TemplateMessage, ConfirmTemplate, MessageAction, CarouselTemplate,
    CarouselColumn, URIAction, PostbackAction )

app = Flask(__name__)

configuration = Configuration(access_token=userdata.get('LINE_CHANNEL_ACCESS_TOKEN'))
handler = WebhookHandler(userdata.get('LINE_CHANNEL_SECRET'))

@app.route("/callback", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)
    return 'OK'


@handler.add(MessageEvent, message=TextMessageContent)
def handle_message(event):
    with ApiClient(configuration) as api_client:
        line_bot_api = MessagingApi(api_client)

        action = event.message.text
        if action == "confirm":
            template = ConfirmTemplate(
                text="你喜歡看影集嗎?",
                actions=[
                    MessageAction(label="是", text="是"),
                    MessageAction(label="否", text="否")
                ]
            )
            reply = TemplateMessage(
                alt_text="這是確認視窗",
                template=template
            )
        elif action == 'carousel':
            carousel_template = CarouselTemplate(
                columns=[
                    CarouselColumn(
                        thumbnail_image_url='https://m.media-amazon.com/images/S/pv-target-images/13e18d3ff20c3e48dbd2bbcad25c08c8049adb21e80110d2cf3c8cf0090bf417.jpg',
                        title='White Collar',
                        text='《雅痞神探》是由美國USA Network電視台播出的刑偵查案電視劇，由Matt Bomer主演。',
                        actions=[
                            URIAction(label='查看詳情', uri='https://zh.wikipedia.org/zh-tw/%E5%A6%99%E8%AD%A6%E8%B3%8A%E6%8E%A2'),
                            MessageAction(label='投票', text='我投White Collar一票!')
                        ]
                    ),
                    CarouselColumn(
                        thumbnail_image_url='https://m.media-amazon.com/images/S/pv-target-images/0020b7dfdbecb99222747371c053f08dde8030309b97bc1b099cfb9e140a2941.jpg',
                        title='House M.D.',
                        text='《怪醫豪斯》是在福斯電視台首播的一部美國醫務電視連續劇，由Hugh Laurie主演。',
                        actions=[
                            URIAction(label='查看詳情', uri='https://zh.wikipedia.org/zh-tw/%E8%B1%AA%E6%96%AF%E5%8C%BB%E7%94%9F'),
                            MessageAction(label='投票', text='我投House M.D.一票!')
                        ]
                    )
                ]
            )
            reply = TemplateMessage(
                alt_text="這是輪播視窗",
                template=carousel_template
            )
        else:
            reply = TextMessage(text = 'Thanks!')

        line_bot_api.reply_message(
            ReplyMessageRequest(
                reply_token=event.reply_token,
                messages=[
                    reply
                ]
            )
        )

ngrok_start() # for colab
if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://71b5-34-66-177-35.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [15/May/2025 09:03:07] "POST /callback HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [15/May/2025 09:03:13] "POST /callback HTTP/1.1" 200 -
